# 🏛️ أتمتة العملية الاكتوارية للتسعير
## Actuarial Pricing Process Automation

### 📌 الهدف:
شرح كيفية أتمتة عملية التسعير الاكتوارية مع توضيح **نقاط التدخل واتخاذ القرار** في كل مرحلة

### 🔄 مراحل العملية:
```
1. جمع البيانات → 2. تنظيف البيانات → 3. التحليل الاستكشافي → 4. تقدير الخسائر
                                   ↓
8. المراقبة ← 7. التسعير النهائي ← 6. حساب الأقساط ← 5. عوامل الخطر
```

## 📦 المرحلة 0: إعداد البيئة

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install -q xgboost lightgbm catboost shap kaggle

In [ ]:
# إستيراد المكتبات
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

print('✅ تم تحميل جميع المكتبات')

---
## 📥 المرحلة 1: جمع البيانات

### 🎯 الهدف:
جمع بيانات المطالبات التاريخية

### 🔧 نقطة القرار:
- ما الفترة الزمنية المناسبة؟
- هل نحتاج بيانات خارجية؟

In [ ]:
# === نقطة قرار: إعدادات جمع البيانات ===
DATA_CONFIG = {
    'source': 'kaggle',
    'period_years': 5,
    'include_external': True
}

print(f'📊 إعدادات جمع البيانات:')
print(f'   - المصدر: {DATA_CONFIG["source"]}')
print(f'   - الفترة: {DATA_CONFIG["period_years"]} سنوات')

In [ ]:
# رفع ملف kaggle.json (من حسابك على Kaggle)
from google.colab import files
print('📤 ارفع ملف kaggle.json:')
uploaded = files.upload()

In [ ]:
# تحميل بيانات المسابقة
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c actuarial-loss-estimation
!unzip -q -o actuarial-loss-estimation.zip
print('✅ تم تحميل البيانات')

In [ ]:
# تحميل البيانات
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'📈 حجم بيانات التدريب: {train.shape}')
print(f'📉 حجم بيانات الاختبار: {test.shape}')
train.head()

---
## 🧹 المرحلة 2: تنظيف البيانات

### 🔧 نقاط القرار:
- كيف نتعامل مع القيم المفقودة؟
- هل نحذف القيم المتطرفة؟

In [ ]:
# تحليل جودة البيانات
print('📋 تقرير جودة البيانات:')
print(f'إجمالي السجلات: {len(train):,}')

missing = train.isnull().sum()
missing_pct = (missing / len(train) * 100).round(2)
missing_report = pd.DataFrame({'عدد المفقود': missing, 'النسبة %': missing_pct})
missing_report[missing_report['عدد المفقود'] > 0]

In [ ]:
# === نقطة قرار: استراتيجية القيم المفقودة ===
MISSING_STRATEGY = {
    'numeric': 'median',       # median, mean, zero
    'categorical': 'mode',     # mode, 'MISSING'
    'threshold_drop': 0.5      # حذف أعمدة > 50% مفقود
}

print('🔧 استراتيجية معالجة القيم المفقودة:')
for k, v in MISSING_STRATEGY.items():
    print(f'   - {k}: {v}')

In [ ]:
# تنظيف البيانات
def clean_data(df, config):
    df = df.copy()
    
    # معالجة الأعمدة الرقمية
    for col in df.select_dtypes(include=[np.number]).columns:
        if df[col].isnull().any():
            if config['numeric'] == 'median':
                df[col] = df[col].fillna(df[col].median())
            elif config['numeric'] == 'mean':
                df[col] = df[col].fillna(df[col].mean())
    
    # معالجة الأعمدة الفئوية
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna('MISSING')
    
    return df

train_clean = clean_data(train, MISSING_STRATEGY)
test_clean = clean_data(test, MISSING_STRATEGY)
print('✅ تم تنظيف البيانات')

---
## 📊 المرحلة 3: التحليل الاستكشافي

### 🔧 نقاط القرار:
- هل التوزيع يحتاج تحويل؟
- ما هي المتغيرات المؤثرة؟

In [ ]:
TARGET = 'UltimateIncurredClaimCost'

# توزيع المتغير المستهدف
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(train_clean[TARGET], bins=50, color='steelblue', edgecolor='white')
axes[0].set_title('التوزيع الأصلي', fontsize=12)

axes[1].hist(np.log1p(train_clean[TARGET]), bins=50, color='coral', edgecolor='white')
axes[1].set_title('بعد التحويل اللوغاريتمي', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# === نقطة قرار: تحويل المتغير المستهدف ===
TARGET_TRANSFORM = 'log'  # 'none', 'log', 'sqrt'

print(f'🎯 قرار: استخدام تحويل {TARGET_TRANSFORM}')

if TARGET_TRANSFORM == 'log':
    y = np.log1p(train_clean[TARGET])
else:
    y = train_clean[TARGET]

---
## 📈 المرحلة 4: تقدير الخسائر (بناء النماذج)

### 🔧 نقاط القرار:
- أي نموذج نستخدم؟
- ما هي المعاملات المثلى؟

In [ ]:
# تحضير الميزات
EXCLUDE_COLS = [TARGET, 'ClaimNumber', 'ClaimDescription', 'AccidentDescription',
                'DateTimeOfAccident', 'DateReported', 'DateOfBirth']

feature_cols = [c for c in train_clean.columns if c not in EXCLUDE_COLS]

# ترميز المتغيرات الفئوية
for col in train_clean[feature_cols].select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    all_vals = pd.concat([train_clean[col], test_clean[col]]).astype(str).unique()
    le.fit(all_vals)
    train_clean[col] = le.transform(train_clean[col].astype(str))
    test_clean[col] = le.transform(test_clean[col].astype(str))

X = train_clean[feature_cols].fillna(-999)
X_test = test_clean[feature_cols].fillna(-999)

print(f'✅ تم تحضير {len(feature_cols)} ميزة')

In [ ]:
# === نقطة قرار: إعدادات النماذج ===
MODEL_CONFIG = {
    'models': ['xgboost', 'lightgbm', 'catboost'],
    'n_folds': 5,
    'ensemble_method': 'average',
    'hyperparams': {
        'max_depth': 6,
        'learning_rate': 0.05,
        'n_estimators': 1000
    }
}

print('🤖 إعدادات النمذجة:')
print(f'   - النماذج: {MODEL_CONFIG["models"]}')
print(f'   - Folds: {MODEL_CONFIG["n_folds"]}')
print(f'   - الدمج: {MODEL_CONFIG["ensemble_method"]}')

In [ ]:
# تدريب النماذج
kf = KFold(n_splits=MODEL_CONFIG['n_folds'], shuffle=True, random_state=42)
results = {}

for model_name in MODEL_CONFIG['models']:
    print(f'\n🚀 تدريب {model_name}...')
    oof = np.zeros(len(X))
    pred = np.zeros(len(X_test))
    
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        
        if model_name == 'xgboost':
            model = xgb.XGBRegressor(n_estimators=1000, max_depth=6, learning_rate=0.05,
                                      random_state=42, verbosity=0)
            model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        elif model_name == 'lightgbm':
            model = lgb.LGBMRegressor(n_estimators=1000, max_depth=6, learning_rate=0.05,
                                       random_state=42, verbose=-1)
            model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                     callbacks=[lgb.early_stopping(50, verbose=False)])
        else:
            model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.05,
                                       random_state=42, verbose=0)
            model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50)
        
        oof[val_idx] = model.predict(X_val)
        pred += model.predict(X_test) / MODEL_CONFIG['n_folds']
        print(f'   Fold {fold+1}: MAE = {mean_absolute_error(y_val, oof[val_idx]):.4f}')
    
    results[model_name] = {'oof': oof, 'pred': pred}
    print(f'   ✅ OOF MAE: {mean_absolute_error(y, oof):.4f}')

---
## ⚖️ المرحلة 5: تحليل عوامل الخطر

### 🔧 نقطة القرار:
- هل العوامل منطقية؟
- هل نحتاج تعديلات؟

In [ ]:
# أهمية الميزات
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importance['feature'][:15], importance['importance'][:15], color='steelblue')
plt.xlabel('الأهمية')
plt.title('أهم 15 عامل خطر')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

---
## 💰 المرحلة 6: حساب الأقساط

### 🔧 نقاط القرار:
- نسبة التحميل؟
- هامش الربح؟

In [ ]:
# === نقطة قرار: معاملات التسعير ===
PRICING_CONFIG = {
    'expense_loading': 0.25,      # المصاريف الإدارية
    'profit_margin': 0.10,        # هامش الربح
    'contingency_margin': 0.05,   # هامش الطوارئ
    'reinsurance_cost': 0.03,     # إعادة التأمين
    'commission_rate': 0.15       # العمولات
}

print('💰 معاملات التسعير:')
for k, v in PRICING_CONFIG.items():
    print(f'   - {k}: {v*100:.0f}%')

total_loading = sum(PRICING_CONFIG.values())
print(f'\n   📊 إجمالي التحميل: {total_loading*100:.0f}%')

In [ ]:
# حساب الأقساط
ensemble_pred = np.mean([results[m]['pred'] for m in results], axis=0)
expected_loss = np.expm1(ensemble_pred)  # تحويل عكسي

premium = expected_loss * (1 + total_loading)

print(f'📊 إحصائيات الأقساط:')
print(f'   - المتوسط: ${np.mean(premium):,.0f}')
print(f'   - الوسيط: ${np.median(premium):,.0f}')

---
## ✅ المرحلة 7: التسعير النهائي

### 🔧 نقاط القرار:
- تعديلات السوق؟
- الحدود القصوى والدنيا؟

In [ ]:
# === نقطة قرار: التعديلات النهائية ===
FINAL_CONFIG = {
    'market_adjustment': 0.95,    # خصم 5% للتنافسية
    'min_premium': 100,
    'max_premium': 1000000
}

final_premium = premium * FINAL_CONFIG['market_adjustment']
final_premium = np.clip(final_premium, FINAL_CONFIG['min_premium'], FINAL_CONFIG['max_premium'])

print('✅ الأقساط النهائية:')
print(f'   - المتوسط: ${np.mean(final_premium):,.0f}')
print(f'   - الوسيط: ${np.median(final_premium):,.0f}')

---
## 📤 المرحلة 8: إنشاء ملف التقديم

In [ ]:
# إنشاء ملف التقديم
submission = pd.DataFrame({
    'ClaimNumber': test_clean['ClaimNumber'],
    'UltimateIncurredClaimCost': np.maximum(expected_loss, 0)
})

submission.to_csv('submission.csv', index=False)
print('✅ تم حفظ: submission.csv')
submission.head()

In [ ]:
# تحميل الملف
from google.colab import files
files.download('submission.csv')
print('📥 تم تحميل الملف')

---
## 📋 ملخص العملية الاكتوارية

| المرحلة | الهدف | نقاط القرار |
|---------|-------|-------------|
| 1. جمع البيانات | جمع البيانات التاريخية | المصادر، الفترة |
| 2. تنظيف البيانات | ضمان الجودة | المفقود، المتطرف |
| 3. التحليل | فهم الأنماط | التحويلات |
| 4. تقدير الخسائر | بناء النماذج | النموذج، المعاملات |
| 5. عوامل الخطر | تحديد المؤثرات | الأوزان |
| 6. حساب الأقساط | التحويل لأقساط | التحميلات |
| 7. التسعير النهائي | الاعتماد | تعديلات السوق |

### 🎉 تم بناء نظام أتمتة متكامل للتسعير الاكتواري!